In [140]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

In [141]:
read_file = pd.read_excel ("MSN Codes.xlsx")
read_file.to_csv ("MSN Codes.csv", index = None, header=True)
read_file = pd.read_excel ("Investment_Data_Testing.xlsx")
read_file.to_csv ("Investment_Data_Testing.csv", index = None, header=True)
# read_file = pd.read_excel ("Investment_Data_2020.xlsx")
# read_file.to_csv ("Investment_Data_2020.csv", index = None, header=True)

In [142]:
msn_code = pd.read_csv('MSN Codes.csv')

investment_data = pd.read_csv("Investment_Data_Testing.csv")

In [143]:
msn_code.head()

,MSN,Description,Unit
0,BDFDB,Biomass inputs (feedstock) to the production o...,Billion Btu
1,BDPRP,Biodiesel production,Thousand barrels
2,BFFDB,Biomass inputs (feedstock) to the production o...,Billion Btu
3,BFPRP,Biofuels production,Thousand barrels
4,CLPRB,Coal production,Billion Btu


In [144]:
investment_data.head()

,Unnamed: 0,MSN,StateCode,Year,Amount,State,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
0,0,BDFDB,AK,2015,21.0,Alaska,35.027804,16.0,3345612.0
1,1,BDPRP,AK,2015,4.0,Alaska,35.027804,16.0,3345612.0
2,2,BFFDB,AK,2015,21.0,Alaska,35.027804,16.0,3345612.0
3,3,BFPRP,AK,2015,4.0,Alaska,35.027804,16.0,3345612.0
4,4,CLPRB,AK,2015,17747.0,Alaska,35.027804,16.0,3345612.0


In [145]:
investment_data.columns

Index(['Unnamed: 0', 'MSN', 'StateCode', 'Year', 'Amount', 'State',
       'CO2 Emissions (Mmt)', 'TotalNumberofInvestments',
       'TotalAmountofAssistance'],
      dtype='object')

In [146]:
single_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'CO2 Emissions (Mmt)', 'TotalNumberofInvestments', 'TotalAmountofAssistance'
]].drop_duplicates()
single_variables

,StateCode,Year,State,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
0,AK,2015,Alaska,35.027804,16.0,3345612.0
29,AL,2015,Alabama,119.200889,164.0,1864412.0
58,AR,2015,Arkansas,59.053365,477.0,13453887.0
87,AZ,2015,Arizona,94.978784,65.0,1724853.0
116,CA,2015,California,351.408516,1023.0,30077512.0
...,...,...,...,...,...,...
8958,VT,2020,Vermont,5.420121,23.0,2405926.0
8987,WA,2020,Washington,68.372251,68.0,2509973.0
9016,WI,2020,Wisconsin,87.037384,101.0,9860324.0
9045,WV,2020,West Virginia,76.924478,44.0,1052654.0


In [147]:
multi_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'MSN', 'Amount'
]].drop_duplicates()

multi_variables

,StateCode,Year,State,MSN,Amount
0,AK,2015,Alaska,BDFDB,21.0
1,AK,2015,Alaska,BDPRP,4.0
2,AK,2015,Alaska,BFFDB,21.0
3,AK,2015,Alaska,BFPRP,4.0
4,AK,2015,Alaska,CLPRB,17747.0
...,...,...,...,...,...
9098,WY,2020,Wyoming,WDPRB,4757.0
9099,WY,2020,Wyoming,WDTCB,4757.0
9100,WY,2020,Wyoming,WSTCB,0.0
9101,WY,2020,Wyoming,WWPRB,4757.0


In [148]:
MSN_feature_pivot = multi_variables.pivot(index=['StateCode', 'Year', 'State'], columns='MSN', values='Amount')
# MSN_feature_pivot[MSN_feature_pivot.isna().any(axis=1)]
MSN_feature_pivot

MSN                                        BDFDB  BDPRP  BFFDB  BFPRP  \
StateCode Year State                                                    
AK        2015 Alaska                       21.0    4.0   21.0    4.0   
          2016 Alaska                       27.0    5.0   27.0    5.0   
          2017 Alaska                       29.0    5.0   29.0    5.0   
          2018 Alaska                       15.0    3.0   15.0    3.0   
          2019 Alaska                        0.0    0.0    0.0    0.0   
...                                          ...    ...    ...    ...   
X5        2015 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2016 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2017 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2018 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2019 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   

MSN                                          CLPRB   CLPRK   CLPRP  COPRK  \
StateCode Year State                                                        
AK        2015 Alaska                      17747.0  15.073  1177.0  5.717   
          2016 Alaska                      13942.0  14.957   932.0  5.722   
          2017 Alaska                      14365.0  14.978   959.0  5.723   
          2018 Alaska                      13752.0  15.253   902.0  5.706   
          2019 Alaska                      14867.0  15.252   975.0  5.698   
...                                            ...     ...     ...    ...   
X5        2015 Federal Offshore - Pacific      NaN     NaN     NaN  5.717   
          2016 Federal Offshore - Pacific      NaN     NaN     NaN  5.722   
          2017 Federal Offshore - Pacific      NaN     NaN     NaN  5.723   
          2018 Federal Offshore - Pacific      NaN     NaN     NaN  5.706   
          2019 Federal Offshore - Pacific      NaN     NaN     NaN  5.698   

MSN                                        EMFDB  ENPRP  ...    REPRB  SOTCB  \
StateCode Year State                                     ...                   
AK        2015 Alaska                        0.0    0.0  ...  23807.0    3.0   
          2016 Alaska                        0.0    0.0  ...  25155.0   14.0   
          2017 Alaska                        0.0    0.0  ...  23620.0   20.0   
          2018 Alaska                        0.0    0.0  ...  24158.0   28.0   
          2019 Alaska                        0.0    0.0  ...  22754.0   40.0   
...                                          ...    ...  ...      ...    ...   
X5        2015 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2016 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2017 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2018 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2019 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   

MSN                                            TEPRB     TETCB  WDEXB   WDPRB  \
StateCode Year State                                                            
AK        2015 Alaska                      1429942.0  614242.0    0.0  6865.0   
          2016 Alaska                      1435363.0  593662.0    0.0  7636.0   
          2017 Alaska                      1454196.0  601492.0    0.0  6501.0   
          2018 Alaska                      1410604.0  604644.0    0.0  6910.0   
          2019 Alaska                      1370342.0  605689.0    0.0  6418.0   
...                                              ...       ...    ...     ...   
X5        2015 Federal Offshore - Pacific    65465.0       NaN    NaN     NaN   
          2016 Federal Offshore - Pacific    35156.0       NaN    NaN     NaN   
          2017 Federal Offshore - Pacific    32701.0       NaN    NaN     NaN   
          2018 Federal Offshore - Pacific    27811.0       NaN    NaN     NaN   
          2019 Federal Offshore - P

In [149]:
tidy_data = MSN_feature_pivot.merge(single_variables, left_on=['StateCode', 'Year'], right_on=['StateCode', 'Year'])
import numpy
past_assistance = numpy.array([0])
past_assistance = numpy.append(past_assistance, tidy_data.iloc[:, -1])
past_assistance = numpy.delete(past_assistance, -1)
tidy_data['PastAssistance'] = past_assistance
# print(tidy_data)
tidy_data_wo_na_response = tidy_data[~tidy_data['TotalAmountofAssistance'].isna()].drop(columns=['State'])
#  print(tidy_data_wo_na_response)

df_filtered = tidy_data_wo_na_response[tidy_data_wo_na_response['Year'] > 2015]
# print(df_filtered)

corr_tidy = df_filtered.corr(method = 'spearman', numeric_only=True)
corr_tidy

,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,EMFDB,...,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance,PastAssistance
Year,1.000000,-0.015392,-0.014285,-0.036568,-0.035302,-0.043885,-0.035763,-0.042269,-0.371976,-0.027905,...,0.059913,-0.008936,-0.013325,-0.031628,-0.017068,0.033195,-0.044248,-0.183236,0.081716,0.015246
BDFDB,-0.015392,1.000000,0.997964,0.727810,0.727687,0.094218,0.098753,0.092874,0.042548,0.527365,...,0.033550,0.226576,0.227962,0.470212,0.302692,0.296893,0.436635,0.391756,0.272056,0.296385
BDPRP,-0.014285,0.997964,1.000000,0.730374,0.733074,0.085180,0.095057,0.083549,0.037622,0.531829,...,0.035913,0.226210,0.227649,0.476347,0.304028,0.295189,0.442561,0.385747,0.275877,0.297116
BFFDB,-0.036568,0.727810,0.730374,1.000000,0.998974,-0.012670,0.004713,-0.014333,0.206886,0.920312,...,-0.124307,0.086641,0.092547,0.384284,0.147400,0.514726,0.407053,0.408988,0.283997,0.292994
BFPRP,-0.035302,0.727687,0.733074,0.998974,1.000000,-0.018143,0.002797,-0.019962,0.201435,0.920316,...,-0.121810,0.086817,0.092799,0.388256,0.148608,0.512604,0.410902,0.404532,0.286589,0.293421
CLPRB,-0.043885,0.094218,0.085180,-0.012670,-0.018143,1.000000,0.899673,0.998407,0.537673,0.059888,...,-0.034752,-0.088688,-0.092412,-0.194611,-0.127651,0.094494,0.316855,-0.018491,-0.158565,-0.161914
CLPRK,-0.035763,0.098753,0.095057,0.004713,0.002797,0.899673,1.000000,0.886751,0.505684,0.054481,...,0.007500,-0.004262,-0.007562,-0.125142,-0.036036,-0.020093,0.347867,-0.018646,-0.134595,-0.110459
CLPRP,-0.042269,0.092874,0.083549,-0.014333,-0.019962,0.998407,0.886751,1.000000,0.540372,0.059824,...,-0.037845,-0.097449,-0.101198,-0.203568,-0.138177,0.110221,0.311910,-0.016467,-0.152416,-0.159493
COPRK,-0.371976,0.042548,0.037622,0.206886,0.201435,0.537673,0.505684,0.540372,1.000000,0.244785,...,-0.016247,-0.044779,-0.045228,-0.120308,-0.074678,0.132818,0.326785,0.152086,-0.065639,-0.062574
EMFDB,-0.027905,0.527365,0.531829,0.920312,0.920316,0.059888,0.054481,0.059824,0.244785,1.000000,...,-0.178994,-0.023444,-0.016286,0.287187,0.022547,0.548676,0.381881,0.359246,0.253187,0.250344


In [150]:
print(corr_tidy.loc['WYTCB', 'TotalAmountofAssistance'])
corr_last3 = corr_tidy.iloc[:, -4:]
corr_last3.loc[corr_last3['TotalAmountofAssistance'] > 0.4]


0.07342317793657652


,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance,PastAssistance
REPRB,0.540102,0.485651,0.422870,0.417343
TETCB,0.960769,0.447688,0.407205,0.408219
TotalNumberofInvestments,0.428249,1.000000,0.502771,0.496183
TotalAmountofAssistance,0.333338,0.502771,1.000000,0.529748
PastAssistance,0.335725,0.496183,0.529748,1.000000


1st try: BDFDB, BFPRP, NUETB, REPRB, TETCB, WWPRB
2nd try: REPRB, TETCB, WWPRB, TotalNumberofInvestments, PastAssistance

In [151]:
tidy_response_data = tidy_data_wo_na_response[['StateCode', 'Year', 'TotalAmountofAssistance']]
tidy_response_data

,StateCode,Year,TotalAmountofAssistance
0,AK,2015,3345612.0
1,AK,2016,2247734.0
2,AK,2017,13509662.0
3,AK,2018,18236626.0
4,AK,2019,13734021.0
...,...,...,...
307,WY,2016,1359346.0
308,WY,2017,413987.0
309,WY,2018,573639.0
310,WY,2019,570570.0


In [154]:
tidy_input_data = df_filtered

X_variables = tidy_input_data#.drop(columns=['StateCode', 'Year'])
y_variables = tidy_response_data#.drop(columns=['StateCode', 'Year'])


train_X = X_variables[(X_variables.Year < 2020) & (X_variables.Year > 2015)].drop(columns=['StateCode', 'Year'])
print(train_X)
train_y = y_variables[(y_variables.Year < 2020) & (y_variables.Year > 2015)].drop(columns=['StateCode', 'Year'])
test_X = X_variables[X_variables.Year == 2020].drop(columns=['StateCode', 'Year'])
compare_results = y_variables[y_variables.Year == 2020]

scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)


regr = ElasticNet(random_state=0, l1_ratio=0.5, alpha = 1)
regr.fit(train_X, train_y)
predictions = regr.predict(test_X)

compare_results['predict'] = predictions.tolist()
compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']

print('RSME: ', ((compare_results.predict - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)

from sklearn.metrics import r2_score
print(r2_score(compare_results['TotalAmountofAssistance'], predictions))
compare_results

      BDFDB  BDPRP   BFFDB  BFPRP      CLPRB   CLPRK     CLPRP  COPRK  EMFDB  \
1      27.0    5.0    27.0    5.0    13942.0  14.957     932.0  5.722    0.0   
2      29.0    5.0    29.0    5.0    14365.0  14.978     959.0  5.723    0.0   
3      15.0    3.0    15.0    3.0    13752.0  15.253     902.0  5.706    0.0   
4       0.0    0.0     0.0    0.0    14867.0  15.252     975.0  5.698    0.0   
7    1906.0  351.0  1906.0  351.0   247632.0  25.680    9643.0  5.722    0.0   
..      ...    ...     ...    ...        ...     ...       ...    ...    ...   
304     0.0    0.0     0.0    0.0  2422224.0  25.927   93425.0  5.698    0.0   
307     0.0    0.0     0.0    0.0  5169939.0  17.394  297218.0  5.722    0.0   
308     0.0    0.0     0.0    0.0  5516757.0  17.433  316454.0  5.723    0.0   
309     0.0    0.0     0.0    0.0  5315953.0  17.476  304188.0  5.706    0.0   
310     0.0    0.0     0.0    0.0  4828513.0  17.437  276912.0  5.698    0.0   

     ENPRP  ...  WDEXB     WDPRB     WD

C:\Users\andre\AppData\Local\Temp\ipykernel_23696\647301587.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_results['predict'] = predictions.tolist()
C:\Users\andre\AppData\Local\Temp\ipykernel_23696\647301587.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']


,StateCode,Year,TotalAmountofAssistance,predict,ans_div_predict
5,AK,2020,9567164.0,8.881095e+06,1.077250
11,AL,2020,1174323.0,6.649890e+06,0.176593
17,AR,2020,61208016.0,4.594301e+07,1.332259
23,AZ,2020,23756565.0,2.879851e+07,0.824923
29,CA,2020,58722209.0,7.199771e+07,0.815612
35,CO,2020,6304206.0,8.566484e+06,0.735915
41,CT,2020,491729.0,4.313637e+06,0.113994
53,DE,2020,23750266.0,1.855189e+07,1.280207
59,FL,2020,8467648.0,2.388017e+07,0.354589
65,GA,2020,7307409.0,2.585489e+07,0.282632


In [153]:
X_variables = tidy_input_data#.drop(columns=['StateCode', 'Year'])
y_variables = tidy_response_data#.drop(columns=['StateCode', 'Year'])


train_X = X_variables[(X_variables.Year < 2019) & (X_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])
train_y = y_variables[(y_variables.Year < 2019) & (y_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])
test_X = X_variables[(X_variables.Year == 2019) & (X_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])
compare_results = y_variables[(y_variables.Year == 2019) & (y_variables.StateCode == 'TX')]

# scaler = StandardScaler()
# scaler.fit(train_X)
# train_X = scaler.transform(train_X)
# test_X = scaler.transform(test_X)


regr = ElasticNet(random_state=0)
regr.fit(train_X, train_y)
predictions = regr.predict(test_X)
print(predictions)
compare_results['TotalAmountofAssistance']

# compare_results['predict'] = predictions.tolist()
# compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']

# print('RSME: ', ((compare_results.predict - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)

# from sklearn.metrics import r2_score
# print(r2_score(compare_results['TotalAmountofAssistance'], predictions))
# compare_results

ValueError: Found input variables with inconsistent numbers of samples: [3, 4]

In [ ]:
# BDFDB, BFPRP, NUETB, REPRB, TETCB, WWPRB

train_X = X_variables[(X_variables.Year < 2019) & (X_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])
train_X = train_X[['BDFDB', 'BFPRP', 'NUETB', 'REPRB', 'TETCB', 'WWPRB']]

train_y = y_variables[(y_variables.Year < 2019) & (y_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])

test_X = X_variables[(X_variables.Year == 2019) & (X_variables.StateCode == 'TX')].drop(columns=['StateCode', 'Year'])
test_X = test_X[['BDFDB', 'BFPRP', 'NUETB', 'REPRB', 'TETCB', 'WWPRB']]

compare_results = y_variables[(y_variables.Year == 2019) & (y_variables.StateCode == 'TX')]

scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)


regr = ElasticNet(random_state=0, l1_ratio=0.5, alpha = 1)
regr.fit(train_X, train_y)
predictions = regr.predict(test_X)

compare_results['predict'] = predictions.tolist()
compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']

print('RSME: ', ((compare_results.predict - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)


RSME:  24628120.800950028


C:\Users\andre\AppData\Local\Temp\ipykernel_23696\627497942.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_results['predict'] = predictions.tolist()
C:\Users\andre\AppData\Local\Temp\ipykernel_23696\627497942.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compare_results['ans_div_predict'] = compare_results['TotalAmountofAssistance'] / compare_results['predict']


In [ ]:
#train_X
# train_y
# test_X

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state=0)
regressor.fit(train_X, train_y)

randomforest_prediction = regressor.predict(test_X)

print('RANDOM PRED')
print(randomforest_prediction)

print('ACTUAL ASSISTANCE')
print(compare_results['TotalAmountofAssistance'])
print('=====')

print(r2_score(compare_results.TotalAmountofAssistance, randomforest_prediction))
print('RSME: ', ((randomforest_prediction - compare_results.TotalAmountofAssistance) ** 2).mean() ** .5)

RANDOM PRED
[1.03476538e+07 2.09292862e+07 2.58945568e+07 5.17772567e+07
 3.26243198e+07 9.98208680e+06 1.21595720e+06 8.40461700e+05
 5.33298253e+07 9.69312691e+07 3.40003800e+06 6.67953260e+06
 1.42053220e+06 2.99966030e+06 5.94474500e+06 1.34273290e+06
 2.18405810e+06 4.94754490e+06 3.99525890e+06 2.85413970e+06
 2.04707880e+06 1.25976110e+06 2.93935344e+07 3.21082330e+06
 1.86611688e+07 2.55912760e+06 1.58811606e+08 5.06086500e+05
 9.24708900e+06 1.17400030e+06 3.17358954e+07 3.89183260e+06
 9.85344683e+07 7.41160860e+06 4.66064460e+06 4.50896780e+06
 4.29762690e+06 4.93148470e+06 1.10231500e+06 5.09865499e+07
 3.76604410e+06 2.60535860e+06 7.11901696e+07 6.52018500e+05
 1.14010963e+07 2.30231010e+06 1.11820336e+07 2.77023010e+06
 9.62702200e+05 8.77035700e+05]
ACTUAL ASSISTANCE
4       13734021.0
9         840218.0
14      60034678.0
19      95135122.0
24     158244985.0
29      10796826.0
34       1306178.0
44       1485254.0
49      52936642.0
54       2374660.0
59        329849

C:\Users\andre\AppData\Local\Temp\ipykernel_23696\259843071.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(train_X, train_y)


In [ ]:
from pygam import LinearGAM
gam = LinearGAM(n_splines=10).gridsearch(train_X, train_y)


  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--


AttributeError: module 'numpy' has no attribute 'int'